# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), February 13, 2023**

In [1]:
using DataFrames

In [2]:
using BenchmarkTools

In [3]:
using CategoricalArrays

In [4]:
using PooledArrays

## Performance tips

### Access by column number is faster than by name

In [5]:
x = DataFrame(rand(5, 1000), :auto)
@btime $x[!, 500];
@btime $x.x500;

  2.000 ns (0 allocations: 0 bytes)
  9.000 ns (0 allocations: 0 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [6]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y, z = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  100.944 ms (5999012 allocations: 122.06 MiB)


In [7]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    f_inner(x[!, 1], x[!, 2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    dot(x[!, 1], x[!, 2])
end

@btime f_barrier();
@btime f_inbuilt();

  6.473 ms (34 allocations: 30.52 MiB)
  6.252 ms (34 allocations: 30.52 MiB)


In [8]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y::Vector{Float64}, z::Vector{Float64} = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  6.440 ms (34 allocations: 30.52 MiB)


In general for tall and narrow tables it is often useful to use `Tables.rowtable`, `Tables.columntable` or `Tables.namedtupleiterator` for intermediate processing of data in a type-stable way.

### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between copying vs non-copying data frame creation

In [9]:
function f1()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto, copycols=false) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto, copycols=false) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  25.195 ms (1949524 allocations: 37.40 MiB)
  26.055 ms (1949724 allocations: 45.03 MiB)
  1.436 ms (624 allocations: 7.66 MiB)
  2.702 ms (824 allocations: 15.30 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [10]:
x = DataFrame(rand(10^6, 5), :auto)
y = DataFrame(transpose(1.0:5.0), :auto)
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5), :auto) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  5.662 ms (188 allocations: 38.16 MiB)
  1.270 μs (30 allocations: 1.80 KiB)
  416.080 ns (16 allocations: 256 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [11]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  3.339 ms (7 allocations: 7.63 MiB)
 categorical:
  11.436 ms (4 allocations: 688 bytes)
String
 raw:
  16.443 ms (4 allocations: 528 bytes)
 categorical:
  22.391 ms (4 allocations: 688 bytes)
Union{Missing, Int64}
 raw:
  5.920 ms (4 allocations: 560 bytes)
 categorical:
  16.192 ms (1000004 allocations: 30.52 MiB)
Union{Missing, String}
 raw:
  17.110 ms (4 allocations: 528 bytes)
 categorical:
  25.003 ms (1000004 allocations: 30.52 MiB)


### When aggregating use column selector and prefer integer, categorical, or pooled array grouping variable

In [12]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [13]:
gdf = groupby(df, :x)

Row,x,y
,Char,Int64
1,a,1
2,a,1
3,a,1
4,a,1
5,a,1
6,a,1
7,a,1
8,a,1
9,a,1


In [14]:
@btime combine(v -> sum(v.y), $gdf) # traditional syntax, slow

  38.910 ms (303 allocations: 19.08 MiB)


Row,x,x1
,Char,Int64
1,a,2499156
2,c,2500605
3,b,2501067
4,d,2499172


In [15]:
@btime combine($gdf, :y=>sum) # use column selector

  8.782 ms (218 allocations: 13.33 KiB)


Row,x,y_sum
,Char,Int64
1,a,2499156
2,c,2500605
3,b,2501067
4,d,2499172


In [16]:
transform!(df, :x => categorical => :x);

In [17]:
gdf = groupby(df, :x)

Row,x,y
,Cat…,Int64
1,a,1
2,a,1
3,a,1
4,a,1
5,a,1
6,a,1
7,a,1
8,a,1
9,a,1


In [18]:
@btime combine($gdf, :y=>sum)

  9.114 ms (225 allocations: 13.91 KiB)


Row,x,y_sum
,Cat…,Int64
1,a,2499156
2,b,2501067
3,c,2500605
4,d,2499172


In [19]:
transform!(df, :x => PooledArray{Char} => :x)

Row,x,y
,Char,Int64
1,a,1
2,c,1
3,b,1
4,d,1
5,a,1
6,c,1
7,c,1
8,a,1
9,b,1


In [20]:
gdf = groupby(df, :x)

Row,x,y
,Char,Int64
1,a,1
2,a,1
3,a,1
4,a,1
5,a,1
6,a,1
7,a,1
8,a,1
9,a,1


In [21]:
@btime combine($gdf, :y=>sum)

  9.486 ms (220 allocations: 13.39 KiB)


Row,x,y_sum
,Char,Int64
1,a,2499156
2,c,2500605
3,b,2501067
4,d,2499172


### Use views instead of materializing a new DataFrame

In [22]:
x = DataFrame(rand(100, 1000), :auto)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0523643,0.609618,0.959066,0.940724,0.497786,0.42965,0.913882,0.747651,0.446724,0.0749807,0.293294,0.680345,0.162048,0.159452,0.958113,0.922532,0.834303,0.966029,0.38394,0.625653,0.830014,0.251016,0.357237,0.466607,0.749349,0.126183,0.109127,0.0957731,0.194717,0.355339,0.514552,0.744211,0.123781,0.320412,0.552054,0.482046,0.882937,0.629033,0.709226,0.40338,0.267963,0.149298,0.599479,0.479195,0.173125,0.610714,0.0543737,0.0209455,0.620016,0.679305,0.813113,0.52726,0.678049,0.602801,0.263353,0.140502,0.173345,0.00508686,0.882622,0.671508,0.903111,0.992132,0.130133,0.825957,0.749588,0.413133,0.453115,0.981498,0.670985,0.430595,0.716924,0.111158,0.285703,0.984693,0.408762,0.84116,0.58661,0.376678,0.117205,0.872283,0.373637,0.367935,0.850897,0.338093,0.826562,0.0382447,0.379849,0.622853,0.923399,0.766872,0.364495,0.0472874,0.168554,0.461221,0.923582,0.993141,0.499462,0.632815,0.326624,0.801396,⋯
2,0.996383,0.323279,0.0114092,0.942402,0.987924,0.936681,0.864094,0.932215,0.601533,0.849567,0.896911,0.757683,0.730265,0.688532,0.492183,0.690537,0.0691119,0.930329,0.55779,0.29403,0.601761,0.300509,0.714345,0.228273,0.857088,0.313058,0.236885,0.626102,0.725923,0.133276,0.657448,0.25016,0.303266,0.864351,0.955437,0.939312,0.765406,0.255136,0.325191,0.501739,0.213504,0.861155,0.621882,0.718258,0.225825,0.901666,0.621183,0.782588,0.733544,0.533512,0.818545,0.705379,0.323127,0.812303,0.438585,0.704467,0.316937,0.90005,0.491489,0.144504,0.848915,0.337561,0.462783,0.0895823,0.54517,0.207144,0.558059,0.247815,0.820066,0.858006,0.737671,0.888525,0.809721,0.21849,0.206777,0.919064,0.74504,0.622381,0.560501,0.119611,0.118803,0.102635,0.937137,0.732696,0.257261,0.757876,0.565027,0.848687,0.126224,0.619676,0.266669,0.909082,0.55591,0.713743,0.0896559,0.605725,0.193504,0.285495,0.770703,0.661229,⋯
3,0.141696,0.622605,0.661393,0.1708,0.11548,0.475541,0.737145,0.484979,0.109468,0.508312,0.228798,0.358733,0.610601,0.560448,0.535695,0.59929,0.605758,0.677717,0.154823,0.703207,0.507986,0.0501023,0.911171,0.810978,0.199227,0.446064,0.0572667,0.673722,0.993353,0.610138,0.735754,0.855437,0.81269,0.770168,0.0609744,0.649577,0.80488,0.0434918,0.713666,0.665052,0.200676,0.748503,0.447064,0.151912,0.905773,0.69753,0.635902,0.688135,0.528754,0.381539,0.81904,0.0116928,0.179366,0.785532,0.44845,0.081841,0.84552,0.372275,0.0646395,0.790937,0.356194,0.176315,0.0502913,0.344854,0.0719761,0.987706,0.740029,0.53155,0.833661,0.657139,0.183765,0.68696,0.115185,0.678207,0.843813,0.919121,0.990986,0.471223,0.340498,0.758822,0.534306,0.128232,0.907322,0.885358,0.0772653,0.372709,0.953812,0.705903,0.502916,0.384139,0.768066,0.729694,0.564221,0.966081,0.923246,0.908125,0.990469,0.453515,0.412359,0.600875,⋯
4,0.157635,0.203508,0.95652,0.652674,0.433547,0.0857123,0.430567,0.701038,0.162177,0.789321

In [23]:
@btime $x[1:1, :]

  161.200 μs (2986 allocations: 159.50 KiB)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0523643,0.609618,0.959066,0.940724,0.497786,0.42965,0.913882,0.747651,0.446724,0.0749807,0.293294,0.680345,0.162048,0.159452,0.958113,0.922532,0.834303,0.966029,0.38394,0.625653,0.830014,0.251016,0.357237,0.466607,0.749349,0.126183,0.109127,0.0957731,0.194717,0.355339,0.514552,0.744211,0.123781,0.320412,0.552054,0.482046,0.882937,0.629033,0.709226,0.40338,0.267963,0.149298,0.599479,0.479195,0.173125,0.610714,0.0543737,0.0209455,0.620016,0.679305,0.813113,0.52726,0.678049,0.602801,0.263353,0.140502,0.173345,0.00508686,0.882622,0.671508,0.903111,0.992132,0.130133,0.825957,0.749588,0.413133,0.453115,0.981498,0.670985,0.430595,0.716924,0.111158,0.285703,0.984693,0.408762,0.84116,0.58661,0.376678,0.117205,0.872283,0.373637,0.367935,0.850897,0.338093,0.826562,0.0382447,0.379849,0.622853,0.923399,0.766872,0.364495,0.0472874,0.168554,0.461221,0.923582,0.993141,0.499462,0.632815,0.326624,0.801396,⋯


In [24]:
@btime $x[1, :]

  23.795 ns (0 allocations: 0 bytes)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0523643,0.609618,0.959066,0.940724,0.497786,0.42965,0.913882,0.747651,0.446724,0.0749807,0.293294,0.680345,0.162048,0.159452,0.958113,0.922532,0.834303,0.966029,0.38394,0.625653,0.830014,0.251016,0.357237,0.466607,0.749349,0.126183,0.109127,0.0957731,0.194717,0.355339,0.514552,0.744211,0.123781,0.320412,0.552054,0.482046,0.882937,0.629033,0.709226,0.40338,0.267963,0.149298,0.599479,0.479195,0.173125,0.610714,0.0543737,0.0209455,0.620016,0.679305,0.813113,0.52726,0.678049,0.602801,0.263353,0.140502,0.173345,0.00508686,0.882622,0.671508,0.903111,0.992132,0.130133,0.825957,0.749588,0.413133,0.453115,0.981498,0.670985,0.430595,0.716924,0.111158,0.285703,0.984693,0.408762,0.84116,0.58661,0.376678,0.117205,0.872283,0.373637,0.367935,0.850897,0.338093,0.826562,0.0382447,0.379849,0.622853,0.923399,0.766872,0.364495,0.0472874,0.168554,0.461221,0.923582,0.993141,0.499462,0.632815,0.326624,0.801396,⋯


In [25]:
@btime view($x, 1:1, :)

  23.795 ns (0 allocations: 0 bytes)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0523643,0.609618,0.959066,0.940724,0.497786,0.42965,0.913882,0.747651,0.446724,0.0749807,0.293294,0.680345,0.162048,0.159452,0.958113,0.922532,0.834303,0.966029,0.38394,0.625653,0.830014,0.251016,0.357237,0.466607,0.749349,0.126183,0.109127,0.0957731,0.194717,0.355339,0.514552,0.744211,0.123781,0.320412,0.552054,0.482046,0.882937,0.629033,0.709226,0.40338,0.267963,0.149298,0.599479,0.479195,0.173125,0.610714,0.0543737,0.0209455,0.620016,0.679305,0.813113,0.52726,0.678049,0.602801,0.263353,0.140502,0.173345,0.00508686,0.882622,0.671508,0.903111,0.992132,0.130133,0.825957,0.749588,0.413133,0.453115,0.981498,0.670985,0.430595,0.716924,0.111158,0.285703,0.984693,0.408762,0.84116,0.58661,0.376678,0.117205,0.872283,0.373637,0.367935,0.850897,0.338093,0.826562,0.0382447,0.379849,0.622853,0.923399,0.766872,0.364495,0.0472874,0.168554,0.461221,0.923582,0.993141,0.499462,0.632815,0.326624,0.801396,⋯


In [26]:
@btime $x[1:1, 1:20]

  4.287 μs (51 allocations: 4.28 KiB)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0523643,0.609618,0.959066,0.940724,0.497786,0.42965,0.913882,0.747651,0.446724,0.0749807,0.293294,0.680345,0.162048,0.159452,0.958113,0.922532,0.834303,0.966029,0.38394,0.625653


In [27]:
@btime $x[1, 1:20]

  22.869 ns (0 allocations: 0 bytes)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0523643,0.609618,0.959066,0.940724,0.497786,0.42965,0.913882,0.747651,0.446724,0.0749807,0.293294,0.680345,0.162048,0.159452,0.958113,0.922532,0.834303,0.966029,0.38394,0.625653


In [28]:
@btime view($x, 1:1, 1:20)

  26.807 ns (0 allocations: 0 bytes)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0523643,0.609618,0.959066,0.940724,0.497786,0.42965,0.913882,0.747651,0.446724,0.0749807,0.293294,0.680345,0.162048,0.159452,0.958113,0.922532,0.834303,0.966029,0.38394,0.625653
